## Feature Engineering ##

In [5]:
#Dependencies
from sklearn.base import TransformerMixin
import pandas as pd
import numpy as np

from scipy.io import arff

In the EDA, it is seen that the amount of Cement, Superplaticizer and Age affected the strength of concrete most strongly.\
Ratios of these materials to water are possible avenues for feature engineering.

In [6]:
arff_file = arff.loadarff('../data/cement.arff')
X_full = pd.DataFrame(arff_file[0])

In [7]:
X_full.describe()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


In [8]:
class FeatureEngineer(TransformerMixin):
    def __init__(self):
        return None

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Example feature engineering: log transformation
        X['cement_to_water_ratio'] = X['cement'] / X['water']
        X['coarse_aggregate_to_water_ratio'] = X['coarse_aggregate'] / X['water']
        X['fine_aggregate_to_water_ratio'] = X['fine_aggregate'] / X['water']
        # Impute inf and -inf with 0
        X.replace([np.inf, -np.inf], 0, inplace=True)
        return X

These pairs of features have high correlation, shown in the EDA step.

In [9]:
X_full.describe()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


In [10]:
X_full.shape

(1030, 9)

In [11]:
# Export the data
X_full.to_csv('../data/concrete_engineered.csv', index=False)